In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier

In [27]:
#loading the data
bank_df = pd.read_csv("train.csv", index_col="id")
bank_df = bank_df.drop(['CustomerId', 'Surname'], axis=1)
bank_df = bank_df.sample(frac=1)

In [28]:
# Splitting data into training and testing sets
X = bank_df.drop(["Exited"],axis=1)
y = bank_df.Exited

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=125
)

In [29]:
# Identify numerical and categorical columns
cat_col = [1,2]
num_col = [0,3,4,5,6,7,8,9]



In [30]:
# Transformers for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

In [31]:
# Transformers for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

In [32]:
# Combine pipelines using ColumnTransformer
preproc_pipe = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ],
    remainder="passthrough"
)

In [33]:
# Selecting the best features
KBest = SelectKBest(chi2, k="all")


In [34]:

# Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=125)


In [35]:
# KBest and model pipeline
train_pipe = Pipeline(
    steps=[
        ("KBest", KBest),
        ("RFmodel", model),
    ]
)

In [36]:
# Combining the preprocessing and training pipelines
complete_pipe = Pipeline(
    steps=[
        
        ("preprocessor", preproc_pipe),
        ("train", train_pipe),
    ]
)

In [37]:
# running the complete pipeline
complete_pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  [1, 2])])),
                ('train',
                 Pipeline(steps=[('KBest',
                                  SelectKBest(k='all',
                                              score_func=<function chi2 at 0x000002990E556020>)),
                                 ('RFmodel',
                                  RandomForestClassifier(random_state=125))]))])

In [38]:
# model accuracy
complete_pipe.score(X_test, y_test)

0.8583951041182768

In [39]:
complete_pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  [1, 2])])),
                ('train',
                 Pipeline(steps=[('KBest',
                                  SelectKBest(k='all',
                                              score_func=<function chi2 at 0x000002990E556020>)),
                                 ('RFmodel',
                                  RandomForestClassifier(random_state=125))]))])